Simple Kalman filter demo
=========================

In this demo, we'll build a Kalman filter using the ForneyLab toolbox. The goal is to filter noisy scalar observations.

Let `xin` be the (unknown) state of the system, `xout` the next state, `u` the system input and `y` the noisy observation of `xin`:

\begin{align*}
    x_{out} &= x_{in} + u\\
    y &= x_{in} + n\\
    u &= u_{const}\\
    n &\sim \mathcal{N}(0, V_{noise})\,.
\end{align*}

For simplicity, we assume that `u` is a constant: `u_const`. We further assume that the observations are corrupted by white Gaussian noise, with variance `V_noise`. 

The Kalman filter can be represented as a Forney-style factor graph:

```
                   (u)
            (equ)   |
    (xin) --> = --> + --> (xout)
              |    (add_u_xin)
       (n)--> +(add_n_y)
              |
             (y)
```

The complete Kalman filter is obtained by chaining multiple of these sections, connecting `xout` of the previous stage to `xin` of the next stage. 

Let's start by importing ForneyLab, defining the constants and creating the nodes:

In [1]:
using ForneyLab

# Settings
V_noise = 200.0 # variance of the noise (n)
u_const = 1.0   # fixed value for u

# Create nodes
PriorNode(id=:xin)
TerminalNode(id=:xout)
TerminalNode(GaussianDistribution(m=0.0, V=V_noise), id=:n)
TerminalNode(id=:y)
AdditionNode(id=:add_n_y)
EqualityNode(id=:equ)
AdditionNode(id=:add_u_xin)
TerminalNode(DeltaDistribution(u_const), id=:u);

As you can see, we create terminal and prior nodes for the nodes that hold known values or distributions. We can pass the value of a `TerminalNode` to the constructor, as we do for the `n` and `u` nodes. These nodes will always send out a message with this payload. The `GaussianDistribution` is defined by its mean vector (`m`) and covariance matrix (`V`). The message of the `u` node has zero variance since we have no uncertainty about its value. Further, we have 2 addition nodes and 1 equality constraint node with 3 interfaces.

Now let's connect the nodes by defining the edges, and define a time wrap to connect the sections:

In [2]:
# Create edges
Edge(n(:xin), n(:equ).i[1])
Edge(n(:n), n(:add_n_y).i[:in1])
Edge(n(:y), n(:add_n_y).i[:in2])
Edge(n(:add_n_y).i[:out], n(:equ).i[2])
Edge(n(:equ).i[3], n(:add_u_xin).i[:in2])
Edge(n(:u), n(:add_u_xin).i[:in1])
Edge(n(:add_u_xin).i[:out], n(:xout))

# Define wrap to chain time sections
Wrap(n(:xout), n(:xin)); # Feed posterior estimate to prior

# Inspect the graph
draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 15583334730260928680 -->
 15583334730260928680 
 
 n 
 
<!-- 16778819052361058296 -->
 16778819052361058296 
 
 + 
 add_n_y 
 
<!-- 15583334730260928680->16778819052361058296 -->
 15583334730260928680->16778819052361058296 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 in1 
 1 out 
 
<!-- 8822026875762030866 -->
 8822026875762030866 
 
 = 
 equ 
 
<!-- 16778819052361058296->8822026875762030866 -->
 16778819052361058296->8822026875762030866 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 2 2 
 3 out 
 
<!-- 5771182896397803356 -->
 5771182896397803356 
 
 xout 
 
<!-- 8287718950577571118 -->
 8287718950577571118 
 
 xin 
 
<!-- 5771182896397803356->8287718950577571118 -->
 5771182896397803356->8287718950577571118 
 
 
 
<!-- 8287718950577571118->8822026875762030866 -->
 8287718950577571118->8822026875762030866 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 1 
 1 out 
 
<!-- 17050987348152025307 -->
 17050987348152025307 
 
 u 
 
<!-- 10735041076631420421 -->
 10735041076631420421 
 
 + 
 add_u_xin 
 
<!-- 17050987348152025307->10735041076631420421 -->
 17050987348152025307->10735041076631420421 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 in1 
 1 out 
 
<!-- 4964752719953547162 -->
 4964752719953547162 
 
 y 
 
<!-- 4964752719953547162->16778819052361058296 -->
 4964752719953547162->16778819052361058296 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 2 in2 
 1 out 
 
<!-- 10735041076631420421->5771182896397803356 -->
 10735041076631420421->5771182896397803356 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 1 out 
 3 out 
 
<!-- 8822026875762030866->10735041076631420421 -->
 8822026875762030866->10735041076631420421 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: Any 
 2 in2 
 3 3

Now that the factor graph is complete, we can set the initial message, create the message passing schedule, and create a buffer to hold the results. In addition to the write buffer, which was introduced in the previous demo, we also define a read buffer that reads an input value on each call to `step()`.

In [3]:
# Set initiatal value of xin to get started
n(:xin).value = GaussianDistribution(m=1.0+randn()*sqrt(V_noise), V=V_noise)

# Data
n_samples = 100
noisy_y = [i+randn()*sqrt(V_noise) for i=1:n_samples] # Generate noisy observations of y
y_buffer = [DeltaDistribution(noisy_y[i]) for i=1:n_samples]
attachReadBuffer(n(:y), y_buffer) # TerminalNode y will take value y_buffer[n] in timestep n

# Create output buffer to record the values we wish to observe in every timestep
state = attachWriteBuffer(n(:add_u_xin).i[:out])

# Declare the inference procedure
algo = SumProduct.Algorithm()

show(algo.fields[:schedule])

Message passing schedule (entry: node [interface], rule)


------------------------------------------------------
1: TerminalNode u [1:out], sumProduct! 
2: TerminalNode xin [1:out], sumProduct! 
3: TerminalNode n [1:out], sumProduct! 
4: TerminalNode y [1:out], sumProduct! 
5: AdditionNode add_n_y [3:out], sumProduct! 
6: EqualityNode equ [3:3], sumProduct! 
7: AdditionNode add_u_xin [3:out], sumProduct! 


All set. Now we apply the filter by calculating the message towards `xout`. Then, we move this message to `xin` and repeat the process 100 times. This way, we simulate a forward pass over 100 sections.

In [4]:
# Do forward passes
run(algo)

That was easy!
Let's plot the filter output (mean and standard deviation of `xout`) together with the noisy observations to see if it worked.

In [5]:
using PyPlot
results_mean = Float64[mean(s)[1] for s in state]
results_variance = Float64[var(s)[1,1] for s in state]
plot([1:n_samples], noisy_y, "b*")
plot([1:n_samples], results_mean, "b-")
fill_between([1:n_samples], results_mean-sqrt(results_variance), results_mean+sqrt(results_variance), color="b", alpha=0.3);
grid(true)

ErrorException("Failed to pyimport("matplotlib"): PyPlot will not work until you have a functioning matplotlib module.  PyError (:PyImport_ImportModule) <type 'exceptions.ImportError'>
ImportError('No module named matplotlib',)
")


LoadError: ArgumentError("haskey of NULL PyObject")
while loading In[5], in expression starting on line 4